In [ ]:
import pandas as pd
import json
import re

In [ ]:
df = pd.read_excel('./file/doc.xlsx')
df.head(1)

,namespace,jobName,runtime,type,subType,jobParameters
0,mx.kord.app-id-26220.pro,kord-mx-krb-inm-transorderactordocl1tp-01,kirby3-lts,kirby3,kirby3-lts,"{""configUrl"": ""https://artifactory-gdt.central..."


In [ ]:
# Convertir los valores de la columna jobParameters a cadenas de texto
df['jobParameters'] = df['jobParameters'].astype(str)

# Utiliza json.loads() para cargar el JSON en un diccionario
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {}

df['jobParameters'] = df['jobParameters'].apply(parse_json)

In [ ]:
# Utiliza pd.json_normalize para aplanar la columna jobParameters en un DataFrame
normalized_df = pd.json_normalize(df['jobParameters'])

# Combina el DataFrame original con el DataFrame normalizado
df = pd.concat([df, normalized_df], axis=1)

# Elimina la columna si es necesario
del df['jobParameters']
del df['extraLibs']
del df['mainClass']
del df['repoSecurity']
del df['sparkHistoryEnabled']
del df['dataprocSdkOverrideVersion']

# Muestra el DataFrame resultante
df.head()

,namespace,jobName,runtime,type,subType,configUrl,artifactUrl,dataprocSdkOverrideVersion
0,mx.kord.app-id-26220.pro,kord-mx-krb-inm-transorderactordocl1tp-01,kirby3-lts,kirby3,kirby3-lts,https://artifactory-gdt.central-02.nextgen.igr...,NaN,NaN
1,mx.kord.app-id-26220.pro,kord-mx-spk-trn-migrationtool-01,spark:3.1,spark,3.1,https://artifactory-gdt.central-02.nextgen.igr...,https://artifactory-gdt.central-02.nextgen.igr...,NaN
2,mx.kord.app-id-26220.pro,kord-mx-spk-trn-smartcleaner-01,spark:3.1,spark,3.1,https://artifactory-gdt.central-02.nextgen.igr...,https://artifactory-gdt.central-02.nextgen.igr...,0.4.9-3.1
3,mx.kord.app-id-26220.pro,kord-gl-spk-inm-transactionhistoricmaster-01,kirby3-lts,kirby3,kirby3-lts,https://artifactory-gdt.central-02.nextgen.igr...,NaN,NaN
4,mx.kord.app-id-26220.pro,kord-gl-spk-inm-transsetorderdetailmaster-01,kirby3-lts,kirby3,kirby3-lts,https://artifactory-gdt.central-02.nextgen.igr...,NaN,NaN


In [ ]:
# Define una función para extraer los números con punto decimal de una URL
def extract_version(url):
    if isinstance(url, str):
        matches = re.findall(r'\d+\.\d+\.\d+', url)
        if matches:
            return matches[0]
    return None

# Aplica la función a cada fila de la columna configUrl y guarda los resultados en una nueva columna llamada version
df['version'] = df['configUrl'].apply(extract_version)

# Muestra el DataFrame con la nueva columna
df.head(10)

In [ ]:
df.to_csv('kord.csv', index=False)